# CMake issues<br/> when depending on Kokkos

Damien Lebrun-Grandié and Daniel Arndt  
Oak Ridge National Laboratory

# How to<br> find Kokkos as an external project<br> and load its setting


* What are we trying to do?
* How is it done today and what are the limits of the current practice?
* What's different in the approach we suggest and why do we think it is better
* Who cares?
* What are the risks and the payoffs?


`<target>` has been created by `add_library()` or `add_executable()`

How to use say Boost.Serialization
```CMake
find_package(Boost COMPONENTS serialization)
target_link_libraries(<target> Boost::serialization)
```

```C++
// HelloWorld.cpp
#include <Kokkos_Core.hpp>
#include <cstdio>
#include <typeinfo>

int main(int &argc, char *argv[]) {
    
  Kokkos::initialize(argc, argv);
    
  printf("Hello World on Kokkos execution space %s\n",
          typeid(Kokkos::DefaultExecutionSpace).name ());
    
  Kokkos::parallel_for(15, KOKKOS_LAMBDA (int i) {
      printf("Hello from i = %i\n", i);
    });
    
  Kokkos::finalize();
}
```

```CMake
# CMakeLists.txt
find_package(Kokkos)
add_executable(hello.exe HelloWorld.cpp)
target_link_libraries(hello.exe Kokkos::kokkos)
```

## CMake crash course

```CMake
find_package(<PackageName> [version] [EXACT] [QUIET] [MODULE]
             [REQUIRED] [[COMPONENTS] [components...]]
             [OPTIONAL_COMPONENTS components...]
             [NO_POLICY_SCOPE])
```
* `<PackageName>_FOUND` set to indicate whether the package was found or not  
* package-specific information provided through variables and **imported targets**  
* `REQUIRED` option stops processing with an error message if package cannot be found  
* `[version]` argument requests a version with which the pacakge found should be compatible


The command has two modes by which it searches for packages: "Module" mode and "Config" mode.  
Above signature selects Module mode and falls back to Config mode (unless `MODULE` option present)


### Module mode,
CMake searches for a file called `Find<PackageName>.cmake`.  
If found, file is read and processed by CMake.  
Responsible for finding the package, checking the version, and producing any needed messages.

### Config mode  
CMake searches for a file called `<PackageName>Config.cmake` or `<lower-case-package-name>-config.cmake`  
once found configuration file read and processed by CMake

# How to build Kokkos

Kokkos can be installed via
* generated makefile
* CMake
* TriBITS as part of Trilinos (not considered in this talk)

## Configure

* Using provided bash script that generates the makefile
```bash
<kokkos_path>/generate_makefile.bash \
  --prefix=<prefix> \
  --arch=Volta70,SNB \
  --with-serial \
  --with-openmp \
  --with-cuda \
  --with-cuda-options=enable_lambda
```

* Or using CMake
```bash
cmake \
  -D CMAKE_INSTALL_PREFIX=<prefix> \
  -D KOKKOS_ARCH="Volta70;SNB" \
  -D KOKKOS_ENABLE_SERIAL=ON \
  -D KOKKOS_ENABLE_OPENMP=ON \
  -D KOKKOS_ENABLE_CUDA=ON \
  -D KOKKOS_ENABLE_CUDA_LAMBDA=ON \
  -D CMAKE_CXX_COMPILER=<kokkos_path>/bin/nvcc_wrapper \
  -D CMAKE_BUILD_TYPE=Release \
  <kokkos_path>
```

## Build and install

* As usual

```bash
make # or make -j<NPROCS>
make install
```

# Does it matter if Kokkos was installed one way or the other?

# Differences in content of install directories?

## Current master <a class="external-link" href="https://github.com/kokkos/kokkos/tree/5d6e7fb38e96aec88d2c514e1f9be1cf2b549b57">5d6e7f7</a> (tagged as 2.8.00)

```diff
diff --git a/install_tree_gmake b/install_tree_cmake
index b9dae0a4..e7e934de 100644
--- a/install_tree_gmake
+++ b/install_tree_cmake
@@ -1,7 +1,4 @@
 <prefix>
-|-- Makefile.kokkos
-|-- bin
-|   `-- nvcc_wrapper
 |-- include
 |   |-- Cuda
 |   |   |-- KokkosExp_Cuda_IterateTile.hpp
@@ -87,6 +84,52 @@
 |   |   |-- Kokkos_OpenMP_ViewCopyETIAvail.hpp
 |   |   |-- Kokkos_OpenMP_ViewCopyETIDecl.hpp
 |   |   `-- Kokkos_OpenMP_WorkGraphPolicy.hpp
+|   |-- OpenMPTarget
+|   |   |-- Kokkos_OpenMPTarget_Exec.hpp
+|   |   |-- Kokkos_OpenMPTarget_Parallel.hpp
+|   |   `-- Kokkos_OpenMPTarget_Task.hpp
+|   |-- Qthreads
+|   |   |-- Kokkos_QthreadsExec.hpp
+|   |   |-- Kokkos_Qthreads_Parallel.hpp
+|   |   |-- Kokkos_Qthreads_Task.hpp
+|   |   |-- Kokkos_Qthreads_TaskQueue.hpp
+|   |   `-- Kokkos_Qthreads_TaskQueue_impl.hpp
+|   |-- ROCm
+|   |   |-- KokkosExp_ROCm_IterateTile_Refactor.hpp
+|   |   |-- Kokkos_ROCm_Atomic.hpp
+|   |   |-- Kokkos_ROCm_Config.hpp
+|   |   |-- Kokkos_ROCm_Exec.hpp
+|   |   |-- Kokkos_ROCm_Invoke.hpp
+|   |   |-- Kokkos_ROCm_Join.hpp
+|   |   |-- Kokkos_ROCm_Parallel.hpp
+|   |   |-- Kokkos_ROCm_Reduce.hpp
+|   |   |-- Kokkos_ROCm_ReduceScan.hpp
+|   |   |-- Kokkos_ROCm_Scan.hpp
+|   |   |-- Kokkos_ROCm_Task.hpp
+|   |   |-- Kokkos_ROCm_Tile.hpp
+|   |   |-- Kokkos_ROCm_Vectorization.hpp
+|   |   |-- Kokkos_ROCm_ViewCopyETIAvail.hpp
+|   |   |-- Kokkos_ROCm_ViewCopyETIDecl.hpp
+|   |   `-- hc_math_std.hpp
+|   |-- Serial
+|   |   |-- Kokkos_Serial_ViewCopyETIAvail.hpp
+|   |   `-- Kokkos_Serial_ViewCopyETIDecl.hpp
+|   |-- Threads
+|   |   |-- Kokkos_ThreadsExec.hpp
+|   |   |-- Kokkos_ThreadsTeam.hpp
+|   |   |-- Kokkos_Threads_Parallel.hpp
+|   |   |-- Kokkos_Threads_ViewCopyETIAvail.hpp
+|   |   |-- Kokkos_Threads_ViewCopyETIDecl.hpp
+|   |   `-- Kokkos_Threads_WorkGraphPolicy.hpp
+|   |-- eti
+|   |   |-- Cuda
+|   |   |-- OpenMP
+|   |   |-- ROCm
+|   |   |-- Serial
+|   |   |-- Threads
+|   |   `-- common
+|   |       |-- Kokkos_ViewFillCopyETIAvail_Macros.hpp
+|   |       `-- Kokkos_ViewFillCopyETIDecl_Macros.hpp
 |   `-- impl
 |       |-- KokkosExp_Host_IterateTile.hpp
 |       |-- KokkosExp_ViewMapping.hpp
@@ -143,8 +186,16 @@
 |       `-- Kokkos_Volatile_Load.hpp
 |-- kokkos_generated_settings.cmake
 `-- lib
+    |-- CMake
+    |   `-- Kokkos
+    |       |-- KokkosConfig.cmake
+    |       |-- KokkosTargets-noconfig.cmake
+    |       `-- KokkosTargets.cmake
+    |-- cmake
+    |   `-- Kokkos
+    |       `-- kokkos_generated_settings.cmake
     |-- libkokkos.a
     `-- pkgconfig
         `-- kokkos.pc

-7 directories, 140 files
+22 directories, 176 files
```

## Current develop <a class="external-link" href="https://github.com/kokkos/kokkos/tree/e45bbefa878b5eb004f3110d52dd85ae3b9b0d48">e45bbef</a>
```diff
diff --git a/install_tree_gmake b/install_tree_cmake
index e9537c27..fc4c20e3 100644
--- a/install_tree_gmake
+++ b/install_tree_cmake
@@ -1,7 +1,4 @@
 <prefix>
-|-- Makefile.kokkos
-|-- bin
-|   `-- nvcc_wrapper
 |-- include
 |   |-- Cuda
 |   |   |-- KokkosExp_Cuda_IterateTile.hpp
@@ -24,6 +21,11 @@
 |   |   |-- Kokkos_Cuda_ViewCopyETIDecl.hpp
 |   |   |-- Kokkos_Cuda_WorkGraphPolicy.hpp
 |   |   `-- Kokkos_Cuda_abort.hpp
+|   |-- HPX
+|   |   |-- Kokkos_HPX_Task.hpp
+|   |   |-- Kokkos_HPX_ViewCopyETIAvail.hpp
+|   |   |-- Kokkos_HPX_ViewCopyETIDecl.hpp
+|   |   `-- Kokkos_HPX_WorkGraphPolicy.hpp
 |   |-- KokkosCore_config.h
 |   |-- KokkosExp_MDRangePolicy.hpp
 |   |-- Kokkos_AnonymousSpace.hpp
@@ -93,6 +95,53 @@
 |   |   |-- Kokkos_OpenMP_ViewCopyETIAvail.hpp
 |   |   |-- Kokkos_OpenMP_ViewCopyETIDecl.hpp
 |   |   `-- Kokkos_OpenMP_WorkGraphPolicy.hpp
+|   |-- OpenMPTarget
+|   |   |-- Kokkos_OpenMPTarget_Exec.hpp
+|   |   |-- Kokkos_OpenMPTarget_Parallel.hpp
+|   |   `-- Kokkos_OpenMPTarget_Task.hpp
+|   |-- Qthreads
+|   |   |-- Kokkos_QthreadsExec.hpp
+|   |   |-- Kokkos_Qthreads_Parallel.hpp
+|   |   |-- Kokkos_Qthreads_Task.hpp
+|   |   |-- Kokkos_Qthreads_TaskQueue.hpp
+|   |   `-- Kokkos_Qthreads_TaskQueue_impl.hpp
+|   |-- ROCm
+|   |   |-- KokkosExp_ROCm_IterateTile_Refactor.hpp
+|   |   |-- Kokkos_ROCm_Atomic.hpp
+|   |   |-- Kokkos_ROCm_Config.hpp
+|   |   |-- Kokkos_ROCm_Exec.hpp
+|   |   |-- Kokkos_ROCm_Invoke.hpp
+|   |   |-- Kokkos_ROCm_Join.hpp
+|   |   |-- Kokkos_ROCm_Parallel.hpp
+|   |   |-- Kokkos_ROCm_Reduce.hpp
+|   |   |-- Kokkos_ROCm_ReduceScan.hpp
+|   |   |-- Kokkos_ROCm_Scan.hpp
+|   |   |-- Kokkos_ROCm_Task.hpp
+|   |   |-- Kokkos_ROCm_Tile.hpp
+|   |   |-- Kokkos_ROCm_Vectorization.hpp
+|   |   |-- Kokkos_ROCm_ViewCopyETIAvail.hpp
+|   |   |-- Kokkos_ROCm_ViewCopyETIDecl.hpp
+|   |   `-- hc_math_std.hpp
+|   |-- Serial
+|   |   |-- Kokkos_Serial_ViewCopyETIAvail.hpp
+|   |   `-- Kokkos_Serial_ViewCopyETIDecl.hpp
+|   |-- Threads
+|   |   |-- Kokkos_ThreadsExec.hpp
+|   |   |-- Kokkos_ThreadsTeam.hpp
+|   |   |-- Kokkos_Threads_Parallel.hpp
+|   |   |-- Kokkos_Threads_ViewCopyETIAvail.hpp
+|   |   |-- Kokkos_Threads_ViewCopyETIDecl.hpp
+|   |   `-- Kokkos_Threads_WorkGraphPolicy.hpp
+|   |-- eti
+|   |   |-- Cuda
+|   |   |-- HPX
+|   |   |-- OpenMP
+|   |   |-- ROCm
+|   |   |-- Serial
+|   |   |-- Threads
+|   |   `-- common
+|   |       |-- Kokkos_ViewFillCopyETIAvail_Macros.hpp
+|   |       `-- Kokkos_ViewFillCopyETIDecl_Macros.hpp
 |   `-- impl
 |       |-- KokkosExp_Host_IterateTile.hpp
 |       |-- KokkosExp_ViewMapping.hpp
@@ -170,8 +219,16 @@
 |       `-- Kokkos_Volatile_Load.hpp
 |-- kokkos_generated_settings.cmake
 `-- lib
+    |-- CMake
+    |   `-- Kokkos
+    |       |-- KokkosConfig.cmake
+    |       |-- KokkosTargets-noconfig.cmake
+    |       `-- KokkosTargets.cmake
+    |-- cmake
+    |   `-- Kokkos
+    |       `-- kokkos_generated_settings.cmake
     |-- libkokkos.a
     `-- pkgconfig
         `-- kokkos.pc

-7 directories, 167 files
+24 directories, 207 files
```

# A few observations

* CMake does not install `nvcc_wrapper`

* `<prefix>/lib/CMake/Kokkos/KokkosConfig.cmake` will not be discovered by CMake when `<prefix>` is added to `CMAKE_PREFIX_PATH`

* Duplicated `kokkos_generated_settings.cmake` both in `<prefix>` and in `<prefix>/cmake/Kokkos` is confusing

On UNIX platforms, CMake will constructs the following set of possible installation prefixes for the package

`<prefix>/(lib/<arch>|lib*|share)/cmake/<name>*/`  
`<prefix>/(lib/<arch>|lib*|share)/<name>*/`
`<prefix>/(lib/<arch>|lib*|share)/<name>*/(cmake|CMake)/`  
`<prefix>/<name>*/(lib/<arch>|lib*|share)/cmake/<name>*/`  
`<prefix>/<name>*/(lib/<arch>|lib*|share)/<name>*/`  
`<prefix>/<name>*/(lib/<arch>|lib*|share)/<name>*/(cmake|CMake)/`


`<prefix>/lib/Kokkos/CMake/` will be searched, `<prefix>/lib/CMake/Kokkos` will not!!

Use the (undocumented) `CMAKE_FIND_DEBUG_MODE` to get a more verbose output
```bash
Checking prefix [<prefix>]
Checking file [<prefix>/KokkosConfig.cmake]
Checking file [<prefix>/kokkos-config.cmake]
Checking file [<prefix>/kokkos_generated_settings.cmake/KokkosConfig.cmake]
Checking file [<prefix>/kokkos_generated_settings.cmake/kokkos-config.cmake]
Checking file [<prefix>/lib/cmake/Kokkos/KokkosConfig.cmake]
Checking file [<prefix>/lib/cmake/Kokkos/kokkos-config.cmake]
Checking prefix [/usr/local/nvidia/]
Checking prefix [/usr/local/cuda/]
Checking file [/usr/local/cuda/KokkosConfig.cmake]
Checking file [/usr/local/cuda/kokkos-config.cmake]
Checking prefix [/usr/local/]
Checking file [/usr/local/KokkosConfig.cmake]
Checking file [/usr/local/kokkos-config.cmake]
Checking prefix [/usr/]
Checking file [/usr/KokkosConfig.cmake]
Checking file [/usr/kokkos-config.cmake]
Checking prefix [/]
Checking file [/KokkosConfig.cmake]
Checking file [/kokkos-config.cmake]
Checking prefix [/usr/X11R6/]
Checking prefix [/usr/pkg/]
Checking prefix [/opt/]
Checking file [/opt/KokkosConfig.cmake]
Checking file [/opt/kokkos-config.cmake]
Checking file [/opt/kokkos/KokkosConfig.cmake]
Checking file [/opt/kokkos/kokkos-config.cmake]
Checking file [/opt/kokkos/lib/cmake/Kokkos/KokkosConfig.cmake]
Checking file [/opt/kokkos/lib/cmake/Kokkos/kokkos-config.cmake]
CMake Error at CMakeLists.txt:3 (find_package):
  By not providing "FindKokkos.cmake" in CMAKE_MODULE_PATH this project has
  asked CMake to find a package configuration file provided by "Kokkos", but
  CMake did not find one.

  Could not find a package configuration file provided by "Kokkos" with any
  of the following names:

    KokkosConfig.cmake
    kokkos-config.cmake

  Add the installation prefix of "Kokkos" to CMAKE_PREFIX_PATH or set
  "Kokkos_DIR" to a directory containing one of the above files.  If "Kokkos"
  provides a separate development package or SDK, be sure it has been
  installed.


-- Configuring incomplete, errors occurred!
See also "/scratch/build/CMakeFiles/CMakeOutput.log".
See also "/scratch/build/CMakeFiles/CMakeError.log".
```

# Differences in `kokkos_generated_settings.cmake`?

`Kokkos_generated_settings.cmake` sets a number of variables (44)

`KOKKOS_PATH`
`KOKKOS_GMAKE_DEVICES`
`KOKKOS_GMAKE_ARCH`
`KOKKOS_DEBUG_CMAKE`
`KOKKOS_GMAKE_USE_TPLS`
`KOKKOS_CXX_STANDARD`
`KOKKOS_GMAKE_OPTIONS`
`KOKKOS_GMAKE_CUDA_OPTIONS`
`KOKKOS_GMAKE_TPL_INCLUDE_DIRS`
`KOKKOS_GMAKE_TPL_LIBRARY_DIRS`
`KOKKOS_GMAKE_TPL_LIBRARY_NAMES`
`KOKKOS_HEADERS`
`KOKKOS_HEADERS_IMPL`
`KOKKOS_HEADERS_CUDA`
`KOKKOS_HEADERS_OPENMP`
`KOKKOS_HEADERS_HPX`
`KOKKOS_HEADERS_ROCM`
`KOKKOS_HEADERS_THREADS`
`KOKKOS_HEADERS_QTHREADS`
`KOKKOS_OS`
`KOKKOS_CPP_DEPENDS`
`KOKKOS_LINK_DEPENDS`
`KOKKOS_CXXFLAGS`
`KOKKOS_CPPFLAGS`
`KOKKOS_LDFLAGS`
`KOKKOS_CXXLDFLAGS`
`KOKKOS_LIBS`
`KOKKOS_EXTRA_LIBS`
`KOKKOS_LINK_FLAGS`
`KOKKOS_INTERNAL_USE_CUDA`
`KOKKOS_INTERNAL_USE_OPENMP`
`KOKKOS_INTERNAL_USE_HPX`
`KOKKOS_INTERNAL_USE_PTHREADS`
`KOKKOS_INTERNAL_USE_SERIAL`
`KOKKOS_INTERNAL_USE_ROCM`
`KOKKOS_INTERNAL_USE_HPX`
`KOKKOS_INTERNAL_USE_QTHREADS`
`KOKKOS_SRC`
`KOKKOS_CXX_FLAGS`
`KOKKOS_CPP_FLAGS`
`KOKKOS_LD_FLAGS`
`KOKKOS_LIBS_LIST`
`KOKKOS_EXTRA_LIBS_LIST`
`KOKKOS_LINK_FLAGS`

`KOKKOS_GMAKE_DEVICES`
`KOKKOS_GMAKE_ARCH`
`KOKKOS_GMAKE_OPTIONS`
`KOKKOS_GMAKE_CUDA_OPTIONS`
`KOKKOS_GMAKE_TPL_INCLUDE_DIRS`
`KOKKOS_GMAKE_TPL_LIBRARY_DIRS`
`KOKKOS_GMAKE_TPL_LIBRARY_NAMES`
`KOKKOS_GMAKE_USE_TPLS`

`KOKKOS_DEBUG_CMAKE`

`KOKKOS_OS`
`KOKKOS_PATH`

`KOKKOS_HEADERS`
`KOKKOS_HEADERS_IMPL`
`KOKKOS_HEADERS_CUDA`
`KOKKOS_HEADERS_OPENMP`
`KOKKOS_HEADERS_HPX`
`KOKKOS_HEADERS_ROCM`
`KOKKOS_HEADERS_THREADS`
`KOKKOS_HEADERS_QTHREADS`
`KOKKOS_SRC`

`KOKKOS_CPP_DEPENDS`
`KOKKOS_LINK_DEPENDS`

`KOKKOS_INTERNAL_USE_CUDA`
`KOKKOS_INTERNAL_USE_OPENMP`
`KOKKOS_INTERNAL_USE_HPX`
`KOKKOS_INTERNAL_USE_PTHREADS`
`KOKKOS_INTERNAL_USE_SERIAL`
`KOKKOS_INTERNAL_USE_ROCM`
`KOKKOS_INTERNAL_USE_HPX`
`KOKKOS_INTERNAL_USE_QTHREADS`

`KOKKOS_CXXFLAGS`
`KOKKOS_CPPFLAGS`
`KOKKOS_LDFLAGS`
`KOKKOS_CXXLDFLAGS`
`KOKKOS_LIBS`
`KOKKOS_EXTRA_LIBS`
`KOKKOS_LINK_FLAGS`

`KOKKOS_CXX_FLAGS`
`KOKKOS_CPP_FLAGS`
`KOKKOS_LD_FLAGS`
`KOKKOS_LIBS_LIST`
`KOKKOS_EXTRA_LIBS_LIST`
`KOKKOS_LINK_FLAGS`

`KOKKOS_CXX_STANDARD`

## Current master <a class="external-link" href="https://github.com/kokkos/kokkos/tree/5d6e7fb38e96aec88d2c514e1f9be1cf2b549b57">5d6e7f7</a> (tagged as 2.8.00)

```diff
diff --git a/generated_settings_gmake b/generated_settings_cmake
index 8ed290c9..d64a5337 100644
--- a/generated_settings_gmake
+++ b/generated_settings_cmake
@@ -1,15 +1,15 @@
 #Global Settings used to generate this library
-set(KOKKOS_PATH CACHE FILEPATH "Kokkos installation path" FORCE)
-set(KOKKOS_GMAKE_DEVICES "Serial,OpenMP,Cuda" CACHE STRING "Kokkos devices list" FORCE)
+set(KOKKOS_PATH /opt/kokkos CACHE FILEPATH "Kokkos installation path" FORCE)
+set(KOKKOS_GMAKE_DEVICES "Cuda,OpenMP,Serial" CACHE STRING "Kokkos devices list" FORCE)
 set(KOKKOS_GMAKE_ARCH "Volta70,SNB" CACHE STRING "Kokkos architecture flags" FORCE)
-set(KOKKOS_DEBUG_CMAKE OFF CACHE BOOL "Kokkos debug enabled ?" FORCE)
-set(KOKKOS_GMAKE_USE_TPLS "" CACHE STRING "Kokkos templates list" FORCE)
+set(KOKKOS_DEBUG_CMAKE ON CACHE BOOL "Kokkos debug enabled ?" FORCE)
+set(KOKKOS_GMAKE_USE_TPLS "librt" CACHE STRING "Kokkos templates list" FORCE)
 set(KOKKOS_CXX_STANDARD c++11 CACHE STRING "Kokkos C++ standard" FORCE)
-set(KOKKOS_GMAKE_OPTIONS "" CACHE STRING "Kokkos options" FORCE)
+set(KOKKOS_GMAKE_OPTIONS "disable_dualview_modify_check" CACHE STRING "Kokkos options" FORCE)
 set(KOKKOS_GMAKE_CUDA_OPTIONS "enable_lambda" CACHE STRING "Kokkos Cuda options" FORCE)
 set(KOKKOS_GMAKE_TPL_INCLUDE_DIRS " /usr/local/cuda/include" CACHE STRING "Kokkos TPL include directories" FORCE)
 set(KOKKOS_GMAKE_TPL_LIBRARY_DIRS " /usr/local/cuda/lib64" CACHE STRING "Kokkos TPL library directories" FORCE)
-set(KOKKOS_GMAKE_TPL_LIBRARY_NAMES " dl cudart cuda" CACHE STRING "Kokkos TPL library names" FORCE)
+set(KOKKOS_GMAKE_TPL_LIBRARY_NAMES " dl rt cudart cuda" CACHE STRING "Kokkos TPL library names" FORCE)
 if(NOT DEFINED ENV{NVCC_WRAPPER})
 set(NVCC_WRAPPER /scratch/kokkos/bin/nvcc_wrapper CACHE FILEPATH "Path to command nvcc_wrapper" FORCE)
 else()
@@ -17,10 +17,10 @@ else()
 endif()

 #Source and Header files of Kokkos relative to KOKKOS_PATH
-set(KOKKOS_HEADERS /opt/kokkos/include/Kokkos_UniqueToken.hpp /opt/kokkos/include/Kokkos_NumericTraits.hpp /opt/kokkos/include/Kokkos_Timer.hpp /opt/kokkos/include/Kokkos_Pair.hpp /opt/kokkos/include/Kokkos_ExecPolicy.hpp /opt/kokkos/include/Kokkos_Macros.hpp /opt/kokkos/include/Kokkos_ScratchSpace.hpp /opt/kokkos/include/Kokkos_HBWSpace.hpp /opt/kokkos/include/Kokkos_ROCmSpace.hpp /opt/kokkos/include/Kokkos_ROCm.hpp /opt/kokkos/include/KokkosExp_MDRangePolicy.hpp /opt/kokkos/include/Kokkos_Atomic.hpp /opt/kokkos/include/Kokkos_MemoryPool.hpp /opt/kokkos/include/Kokkos_Threads.hpp /opt/kokkos/include/Kokkos_Layout.hpp /opt/kokkos/include/Kokkos_Serial.hpp /opt/kokkos/include/Kokkos_Qthreads.hpp /opt/kokkos/include/Kokkos_Crs.hpp /opt/kokkos/include/Kokkos_Parallel_Reduce.hpp /opt/kokkos/include/Kokkos_Parallel.hpp /opt/kokkos/include/Kokkos_Cuda.hpp /opt/kokkos/include/Kokkos_View.hpp /opt/kokkos/include/Kokkos_TaskPolicy.hpp /opt/kokkos/include/Kokkos_CudaSpace.hpp /opt/kokkos/include/Kokkos_Core.hpp /opt/kokkos/include/Kokkos_WorkGraphPolicy.hpp /opt/kokkos/include/Kokkos_Vectorization.hpp /opt/kokkos/include/Kokkos_OpenMP.hpp /opt/kokkos/include/Kokkos_hwloc.hpp /opt/kokkos/include/Kokkos_Array.hpp /opt/kokkos/include/Kokkos_MasterLock.hpp /opt/kokkos/include/Kokkos_OpenMPTargetSpace.hpp /opt/kokkos/include/Kokkos_Profiling_ProfileSection.hpp /opt/kokkos/include/Kokkos_CopyViews.hpp /opt/kokkos/include/Kokkos_OpenMPTarget.hpp /opt/kokkos/include/Kokkos_Core_fwd.hpp /opt/kokkos/include/Kokkos_Complex.hpp /opt/kokkos/include/Kokkos_HostSpace.hpp /opt/kokkos/include/Kokkos_Concepts.hpp /opt/kokkos/include/Kokkos_TaskScheduler.hpp /opt/kokkos/include/Kokkos_AnonymousSpace.hpp /opt/kokkos/include/Kokkos_MemoryTraits.hpp /opt/kokkos/include/impl/Kokkos_Timer.hpp /opt/kokkos/include/impl/Kokkos_Error.hpp /opt/kokkos/include/impl/Kokkos_ViewFillCopyETIDecl.hpp /opt/kokkos/include/impl/Kokkos_Traits.hpp /opt/kokkos/include/impl/Kokkos_ViewMapping.hpp /opt/kokkos/include/impl/Kokkos_Spinwait.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Compare_Exchange_Strong.hpp /opt/kokkos/include/impl/Kokkos_AnalyzePolicy.hpp /opt/kokkos/include/impl/Kokkos_FunctorAdapter.hpp /opt/kokkos/include/impl/Kokkos_Volatile_Load.hpp /opt/kokkos/include/impl/Kokkos_FunctorAnalysis.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Add.hpp /opt/kokkos/include/impl/Kokkos_ViewFillCopyETIAvail.hpp /opt/kokkos/include/impl/Kokkos_Profiling_Interface.hpp /opt/kokkos/include/impl/Kokkos_ViewArray.hpp /opt/kokkos/include/impl/Kokkos_HostBarrier.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Windows.hpp /opt/kokkos/include/impl/Kokkos_CPUDiscovery.hpp /opt/kokkos/include/impl/KokkosExp_Host_IterateTile.hpp /opt/kokkos/include/impl/Kokkos_Profiling_DeviceInfo.hpp /opt/kokkos/include/impl/Kokkos_SharedAlloc.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Generic.hpp /opt/kokkos/include/impl/Kokkos_HostThreadTeam.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Assembly.hpp /opt/kokkos/include/impl/Kokkos_ClockTic.hpp /opt/kokkos/include/impl/Kokkos_ViewTile.hpp /opt/kokkos/include/impl/Kokkos_Tags.hpp /opt/kokkos/include/impl/Kokkos_ConcurrentBitset.hpp /opt/kokkos/include/impl/Kokkos_Utilities.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Increment.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Decrement.hpp /opt/kokkos/include/impl/Kokkos_StaticAssert.hpp /opt/kokkos/include/impl/Kokkos_BitOps.hpp /opt/kokkos/include/impl/Kokkos_Memory_Fence.hpp /opt/kokkos/include/impl/KokkosExp_ViewMapping.hpp /opt/kokkos/include/impl/Kokkos_ViewUniformType.hpp /opt/kokkos/include/impl/Kokkos_OldMacros.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Or.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_And.hpp /opt/kokkos/include/impl/Kokkos_TaskQueue_impl.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Sub.hpp /opt/kokkos/include/impl/Kokkos_Serial_WorkGraphPolicy.hpp /opt/kokkos/include/impl/Kokkos_TaskQueue.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Exchange.hpp /opt/kokkos/include/impl/Kokkos_ViewLayoutTiled.hpp /opt/kokkos/include/impl/Kokkos_PhysicalLayout.hpp /opt/kokkos/include/impl/Kokkos_ViewCtor.hpp /opt/kokkos/include/impl/Kokkos_Atomic_View.hpp /opt/kokkos/include/impl/Kokkos_Serial_Task.hpp /opt/kokkos/include/Kokkos_Functional.hpp /opt/kokkos/include/Kokkos_StaticCrsGraph.hpp /opt/kokkos/include/Kokkos_ErrorReporter.hpp /opt/kokkos/include/Kokkos_OffsetView.hpp /opt/kokkos/include/Kokkos_DualView.hpp /opt/kokkos/include/Kokkos_Vector.hpp /opt/kokkos/include/Kokkos_ScatterView.hpp /opt/kokkos/include/Kokkos_Bitset.hpp /opt/kokkos/include/Kokkos_UnorderedMap.hpp /opt/kokkos/include/Kokkos_DynamicView.hpp /opt/kokkos/include/Kokkos_DynRankView.hpp /opt/kokkos/include/impl/Kokkos_StaticCrsGraph_factory.hpp /opt/kokkos/include/impl/Kokkos_Functional_impl.hpp /opt/kokkos/include/impl/Kokkos_Bitset_impl.hpp /opt/kokkos/include/impl/Kokkos_UnorderedMap_impl.hpp /opt/kokkos/include/Kokkos_Random.hpp /opt/kokkos/include/Kokkos_Sort.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Task.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_View.hpp /opt/kokkos/include/Cuda/Kokkos_CudaExec.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Team.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIDecl.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Version_9_8_Compatibility.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_UniqueToken.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_abort.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Parallel.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Locks.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_WorkGraphPolicy.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIAvail.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Error.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Vectorization.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile_Refactor.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Alloc.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ReduceScan.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Internal.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Task.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Team.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIDecl.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Parallel.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_WorkGraphPolicy.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIAvail.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Exec.hpp CACHE STRING "Kokkos headers list" FORCE)
+set(KOKKOS_HEADERS /opt/kokkos/include/Kokkos_Serial.hpp /opt/kokkos/include/Kokkos_ExecPolicy.hpp /opt/kokkos/include/Kokkos_OpenMPTargetSpace.hpp /opt/kokkos/include/Kokkos_OpenMPTarget.hpp /opt/kokkos/include/Kokkos_Macros.hpp /opt/kokkos/include/Kokkos_Qthreads.hpp /opt/kokkos/include/Kokkos_Core_fwd.hpp /opt/kokkos/include/Kokkos_Pair.hpp /opt/kokkos/include/Kokkos_UniqueToken.hpp /opt/kokkos/include/Kokkos_Profiling_ProfileSection.hpp /opt/kokkos/include/Kokkos_OpenMP.hpp /opt/kokkos/include/Kokkos_Vectorization.hpp /opt/kokkos/include/Kokkos_Atomic.hpp /opt/kokkos/include/Kokkos_Crs.hpp /opt/kokkos/include/Kokkos_CudaSpace.hpp /opt/kokkos/include/Kokkos_ScratchSpace.hpp /opt/kokkos/include/Kokkos_CopyViews.hpp /opt/kokkos/include/Kokkos_Parallel_Reduce.hpp /opt/kokkos/include/Kokkos_hwloc.hpp /opt/kokkos/include/Kokkos_ROCmSpace.hpp /opt/kokkos/include/Kokkos_Core.hpp /opt/kokkos/include/Kokkos_HBWSpace.hpp /opt/kokkos/include/Kokkos_TaskScheduler.hpp /opt/kokkos/include/Kokkos_Cuda.hpp /opt/kokkos/include/Kokkos_AnonymousSpace.hpp /opt/kokkos/include/KokkosExp_MDRangePolicy.hpp /opt/kokkos/include/Kokkos_Parallel.hpp /opt/kokkos/include/Kokkos_Complex.hpp /opt/kokkos/include/Kokkos_HostSpace.hpp /opt/kokkos/include/Kokkos_View.hpp /opt/kokkos/include/Kokkos_MasterLock.hpp /opt/kokkos/include/Kokkos_Concepts.hpp /opt/kokkos/include/Kokkos_MemoryPool.hpp /opt/kokkos/include/Kokkos_TaskPolicy.hpp /opt/kokkos/include/Kokkos_NumericTraits.hpp /opt/kokkos/include/Kokkos_Timer.hpp /opt/kokkos/include/Kokkos_WorkGraphPolicy.hpp /opt/kokkos/include/Kokkos_Array.hpp /opt/kokkos/include/Kokkos_Threads.hpp /opt/kokkos/include/Kokkos_Layout.hpp /opt/kokkos/include/Kokkos_ROCm.hpp /opt/kokkos/include/Kokkos_MemoryTraits.hpp /opt/kokkos/include/impl/Kokkos_ClockTic.hpp /opt/kokkos/include/impl/Kokkos_ViewLayoutTiled.hpp /opt/kokkos/include/impl/Kokkos_Profiling_DeviceInfo.hpp /opt/kokkos/include/impl/Kokkos_ViewTile.hpp /opt/kokkos/include/impl/Kokkos_ConcurrentBitset.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Sub.hpp /opt/kokkos/include/impl/Kokkos_Serial_Task.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Windows.hpp /opt/kokkos/include/impl/Kokkos_HostThreadTeam.hpp /opt/kokkos/include/impl/Kokkos_FunctorAnalysis.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_And.hpp /opt/kokkos/include/impl/KokkosExp_ViewMapping.hpp /opt/kokkos/include/impl/Kokkos_ViewFillCopyETIAvail.hpp /opt/kokkos/include/impl/Kokkos_SharedAlloc.hpp /opt/kokkos/include/impl/Kokkos_Volatile_Load.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Add.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Generic.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Compare_Exchange_Strong.hpp /opt/kokkos/include/impl/Kokkos_FunctorAdapter.hpp /opt/kokkos/include/impl/Kokkos_Serial_WorkGraphPolicy.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Assembly.hpp /opt/kokkos/include/impl/Kokkos_ViewUniformType.hpp /opt/kokkos/include/impl/Kokkos_ViewCtor.hpp /opt/kokkos/include/impl/Kokkos_TaskQueue.hpp /opt/kokkos/include/impl/Kokkos_CPUDiscovery.hpp /opt/kokkos/include/impl/Kokkos_ViewArray.hpp /opt/kokkos/include/impl/Kokkos_PhysicalLayout.hpp /opt/kokkos/include/impl/Kokkos_HostBarrier.hpp /opt/kokkos/include/impl/Kokkos_Memory_Fence.hpp /opt/kokkos/include/impl/Kokkos_TaskQueue_impl.hpp /opt/kokkos/include/impl/Kokkos_Error.hpp /opt/kokkos/include/impl/Kokkos_Atomic_View.hpp /opt/kokkos/include/impl/Kokkos_Profiling_Interface.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Exchange.hpp /opt/kokkos/include/impl/KokkosExp_Host_IterateTile.hpp /opt/kokkos/include/impl/Kokkos_OldMacros.hpp /opt/kokkos/include/impl/Kokkos_StaticAssert.hpp /opt/kokkos/include/impl/Kokkos_BitOps.hpp /opt/kokkos/include/impl/Kokkos_Traits.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Increment.hpp /opt/kokkos/include/impl/Kokkos_Tags.hpp /opt/kokkos/include/impl/Kokkos_Timer.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Decrement.hpp /opt/kokkos/include/impl/Kokkos_ViewMapping.hpp /opt/kokkos/include/impl/Kokkos_Spinwait.hpp /opt/kokkos/include/impl/Kokkos_Utilities.hpp /opt/kokkos/include/impl/Kokkos_AnalyzePolicy.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Or.hpp /opt/kokkos/include/impl/Kokkos_ViewFillCopyETIDecl.hpp /opt/kokkos/include/Kokkos_DynRankView.hpp /opt/kokkos/include/Kokkos_Vector.hpp /opt/kokkos/include/Kokkos_StaticCrsGraph.hpp /opt/kokkos/include/Kokkos_Functional.hpp /opt/kokkos/include/Kokkos_DualView.hpp /opt/kokkos/include/Kokkos_DynamicView.hpp /opt/kokkos/include/Kokkos_Bitset.hpp /opt/kokkos/include/Kokkos_UnorderedMap.hpp /opt/kokkos/include/Kokkos_ScatterView.hpp /opt/kokkos/include/Kokkos_ErrorReporter.hpp /opt/kokkos/include/Kokkos_OffsetView.hpp /opt/kokkos/include/impl/Kokkos_StaticCrsGraph_factory.hpp /opt/kokkos/include/impl/Kokkos_Functional_impl.hpp /opt/kokkos/include/impl/Kokkos_Bitset_impl.hpp /opt/kokkos/include/impl/Kokkos_UnorderedMap_impl.hpp /opt/kokkos/include/Kokkos_Random.hpp /opt/kokkos/include/Kokkos_Sort.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_WorkGraphPolicy.hpp /opt/kokkos/include/Cuda/Kokkos_CudaExec.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Team.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Alloc.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIAvail.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIDecl.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Version_9_8_Compatibility.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_UniqueToken.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Task.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_abort.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ReduceScan.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Vectorization.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Locks.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile_Refactor.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Error.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Parallel.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_View.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Internal.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Task.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Exec.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Parallel.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_WorkGraphPolicy.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Team.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIAvail.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIDecl.hpp CACHE STRING "Kokkos headers list" FORCE)
 set(KOKKOS_HEADERS_IMPL CACHE STRING "Kokkos headers impl list" FORCE)
-set(KOKKOS_HEADERS_CUDA /opt/kokkos/include/Cuda/Kokkos_Cuda_Task.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_View.hpp /opt/kokkos/include/Cuda/Kokkos_CudaExec.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Team.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIDecl.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Version_9_8_Compatibility.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_UniqueToken.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_abort.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Parallel.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Locks.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_WorkGraphPolicy.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIAvail.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Error.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Vectorization.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile_Refactor.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Alloc.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ReduceScan.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Internal.hpp CACHE STRING "Kokkos headers Cuda list" FORCE)
-set(KOKKOS_HEADERS_OPENMP /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Task.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Team.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIDecl.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Parallel.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_WorkGraphPolicy.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIAvail.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Exec.hpp CACHE STRING "Kokkos headers OpenMP list" FORCE)
+set(KOKKOS_HEADERS_CUDA /opt/kokkos/include/Cuda/Kokkos_Cuda_WorkGraphPolicy.hpp /opt/kokkos/include/Cuda/Kokkos_CudaExec.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Team.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Alloc.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIAvail.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIDecl.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Version_9_8_Compatibility.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_UniqueToken.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Task.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_abort.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ReduceScan.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Vectorization.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Locks.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile_Refactor.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Error.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Parallel.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_View.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Internal.hpp CACHE STRING "Kokkos headers Cuda list" FORCE)
+set(KOKKOS_HEADERS_OPENMP /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Task.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Exec.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Parallel.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_WorkGraphPolicy.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Team.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIAvail.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIDecl.hpp CACHE STRING "Kokkos headers OpenMP list" FORCE)
 set(KOKKOS_HEADERS_ROCM CACHE STRING "Kokkos headers ROCm list" FORCE)
 set(KOKKOS_HEADERS_THREADS CACHE STRING "Kokkos headers Threads list" FORCE)
 set(KOKKOS_HEADERS_QTHREADS CACHE STRING "Kokkos headers QThreads list" FORCE)
@@ -29,12 +29,12 @@ set(KOKKOS_HEADERS_QTHREADS CACHE STRING "Kokkos headers QThreads list" FORCE)
 set(KOKKOS_OS Linux CACHE STRING "" FORCE)
 set(KOKKOS_CPP_DEPENDS KokkosCore_config.h /opt/kokkos/include/Kokkos_Serial.hpp /opt/kokkos/include/Kokkos_ExecPolicy.hpp /opt/kokkos/include/Kokkos_OpenMPTargetSpace.hpp /opt/kokkos/include/Kokkos_OpenMPTarget.hpp /opt/kokkos/include/Kokkos_Macros.hpp /opt/kokkos/include/Kokkos_Qthreads.hpp /opt/kokkos/include/Kokkos_Core_fwd.hpp /opt/kokkos/include/Kokkos_Pair.hpp /opt/kokkos/include/Kokkos_UniqueToken.hpp /opt/kokkos/include/Kokkos_Profiling_ProfileSection.hpp /opt/kokkos/include/Kokkos_OpenMP.hpp /opt/kokkos/include/Kokkos_Vectorization.hpp /opt/kokkos/include/Kokkos_Atomic.hpp /opt/kokkos/include/Kokkos_Crs.hpp /opt/kokkos/include/Kokkos_CudaSpace.hpp /opt/kokkos/include/Kokkos_ScratchSpace.hpp /opt/kokkos/include/Kokkos_CopyViews.hpp /opt/kokkos/include/Kokkos_Parallel_Reduce.hpp /opt/kokkos/include/Kokkos_hwloc.hpp /opt/kokkos/include/Kokkos_ROCmSpace.hpp /opt/kokkos/include/Kokkos_Core.hpp /opt/kokkos/include/Kokkos_HBWSpace.hpp /opt/kokkos/include/Kokkos_TaskScheduler.hpp /opt/kokkos/include/Kokkos_Cuda.hpp /opt/kokkos/include/Kokkos_AnonymousSpace.hpp /opt/kokkos/include/KokkosExp_MDRangePolicy.hpp /opt/kokkos/include/Kokkos_Parallel.hpp /opt/kokkos/include/Kokkos_Complex.hpp /opt/kokkos/include/Kokkos_HostSpace.hpp /opt/kokkos/include/Kokkos_View.hpp /opt/kokkos/include/Kokkos_MasterLock.hpp /opt/kokkos/include/Kokkos_Concepts.hpp /opt/kokkos/include/Kokkos_MemoryPool.hpp /opt/kokkos/include/Kokkos_TaskPolicy.hpp /opt/kokkos/include/Kokkos_NumericTraits.hpp /opt/kokkos/include/Kokkos_Timer.hpp /opt/kokkos/include/Kokkos_WorkGraphPolicy.hpp /opt/kokkos/include/Kokkos_Array.hpp /opt/kokkos/include/Kokkos_Threads.hpp /opt/kokkos/include/Kokkos_Layout.hpp /opt/kokkos/include/Kokkos_ROCm.hpp /opt/kokkos/include/Kokkos_MemoryTraits.hpp /opt/kokkos/include/impl/Kokkos_ClockTic.hpp /opt/kokkos/include/impl/Kokkos_ViewLayoutTiled.hpp /opt/kokkos/include/impl/Kokkos_Profiling_DeviceInfo.hpp /opt/kokkos/include/impl/Kokkos_ViewTile.hpp /opt/kokkos/include/impl/Kokkos_ConcurrentBitset.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Sub.hpp /opt/kokkos/include/impl/Kokkos_Serial_Task.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Windows.hpp /opt/kokkos/include/impl/Kokkos_HostThreadTeam.hpp /opt/kokkos/include/impl/Kokkos_FunctorAnalysis.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_And.hpp /opt/kokkos/include/impl/KokkosExp_ViewMapping.hpp /opt/kokkos/include/impl/Kokkos_ViewFillCopyETIAvail.hpp /opt/kokkos/include/impl/Kokkos_SharedAlloc.hpp /opt/kokkos/include/impl/Kokkos_Volatile_Load.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Add.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Generic.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Compare_Exchange_Strong.hpp /opt/kokkos/include/impl/Kokkos_FunctorAdapter.hpp /opt/kokkos/include/impl/Kokkos_Serial_WorkGraphPolicy.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Assembly.hpp /opt/kokkos/include/impl/Kokkos_ViewUniformType.hpp /opt/kokkos/include/impl/Kokkos_ViewCtor.hpp /opt/kokkos/include/impl/Kokkos_TaskQueue.hpp /opt/kokkos/include/impl/Kokkos_CPUDiscovery.hpp /opt/kokkos/include/impl/Kokkos_ViewArray.hpp /opt/kokkos/include/impl/Kokkos_PhysicalLayout.hpp /opt/kokkos/include/impl/Kokkos_HostBarrier.hpp /opt/kokkos/include/impl/Kokkos_Memory_Fence.hpp /opt/kokkos/include/impl/Kokkos_TaskQueue_impl.hpp /opt/kokkos/include/impl/Kokkos_Error.hpp /opt/kokkos/include/impl/Kokkos_Atomic_View.hpp /opt/kokkos/include/impl/Kokkos_Profiling_Interface.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Exchange.hpp /opt/kokkos/include/impl/KokkosExp_Host_IterateTile.hpp /opt/kokkos/include/impl/Kokkos_OldMacros.hpp /opt/kokkos/include/impl/Kokkos_StaticAssert.hpp /opt/kokkos/include/impl/Kokkos_BitOps.hpp /opt/kokkos/include/impl/Kokkos_Traits.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Increment.hpp /opt/kokkos/include/impl/Kokkos_Tags.hpp /opt/kokkos/include/impl/Kokkos_Timer.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Decrement.hpp /opt/kokkos/include/impl/Kokkos_ViewMapping.hpp /opt/kokkos/include/impl/Kokkos_Spinwait.hpp /opt/kokkos/include/impl/Kokkos_Utilities.hpp /opt/kokkos/include/impl/Kokkos_AnalyzePolicy.hpp /opt/kokkos/include/impl/Kokkos_Atomic_Fetch_Or.hpp /opt/kokkos/include/impl/Kokkos_ViewFillCopyETIDecl.hpp /opt/kokkos/include/Kokkos_DynRankView.hpp /opt/kokkos/include/Kokkos_Vector.hpp /opt/kokkos/include/Kokkos_StaticCrsGraph.hpp /opt/kokkos/include/Kokkos_Functional.hpp /opt/kokkos/include/Kokkos_DualView.hpp /opt/kokkos/include/Kokkos_DynamicView.hpp /opt/kokkos/include/Kokkos_Bitset.hpp /opt/kokkos/include/Kokkos_UnorderedMap.hpp /opt/kokkos/include/Kokkos_ScatterView.hpp /opt/kokkos/include/Kokkos_ErrorReporter.hpp /opt/kokkos/include/Kokkos_OffsetView.hpp /opt/kokkos/include/impl/Kokkos_StaticCrsGraph_factory.hpp /opt/kokkos/include/impl/Kokkos_Functional_impl.hpp /opt/kokkos/include/impl/Kokkos_Bitset_impl.hpp /opt/kokkos/include/impl/Kokkos_UnorderedMap_impl.hpp /opt/kokkos/include/Kokkos_Random.hpp /opt/kokkos/include/Kokkos_Sort.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_WorkGraphPolicy.hpp /opt/kokkos/include/Cuda/Kokkos_CudaExec.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Team.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Alloc.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIAvail.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ViewCopyETIDecl.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Version_9_8_Compatibility.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_UniqueToken.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Task.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_abort.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_ReduceScan.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Vectorization.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Locks.hpp /opt/kokkos/include/Cuda/KokkosExp_Cuda_IterateTile_Refactor.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Error.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Parallel.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_View.hpp /opt/kokkos/include/Cuda/Kokkos_Cuda_Internal.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Task.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Exec.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Parallel.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_WorkGraphPolicy.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_Team.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIAvail.hpp /opt/kokkos/include/OpenMP/Kokkos_OpenMP_ViewCopyETIDecl.hpp CACHE STRING "" FORCE)
 set(KOKKOS_LINK_DEPENDS libkokkos.a CACHE STRING "" FORCE)
-set(KOKKOS_CXXFLAGS -I./ -I/opt/kokkos/include -I/opt/kokkos/include -I/opt/kokkos/include -I/opt/kokkos/include/eti --std=c++11 -Xcudafe --diag_suppress=esa_on_defaulted_function_ignored -expt-extended-lambda -mavx -arch=sm_70 -I/usr/local/cuda/include -Xcompiler -fopenmp CACHE STRING "" FORCE)
+set(KOKKOS_CXXFLAGS --std=c++11 -Xcudafe --diag_suppress=esa_on_defaulted_function_ignored -expt-extended-lambda -mavx -arch=sm_70 -Xcompiler -fopenmp CACHE STRING "" FORCE)
 set(KOKKOS_CPPFLAGS CACHE STRING "" FORCE)
-set(KOKKOS_LDFLAGS -L/opt/kokkos/lib -mavx -arch=sm_70 -L/usr/local/cuda/lib64 -fopenmp CACHE STRING "" FORCE)
-set(KOKKOS_CXXLDFLAGS -L/opt/kokkos/lib -L/usr/local/cuda/lib64 CACHE STRING "" FORCE)
-set(KOKKOS_LIBS -lkokkos -ldl -lcudart -lcuda CACHE STRING "" FORCE)
-set(KOKKOS_EXTRA_LIBS -ldl -lcudart -lcuda CACHE STRING "" FORCE)
+set(KOKKOS_LDFLAGS -mavx -arch=sm_70 -L/usr/local/cuda/lib64 -fopenmp CACHE STRING "" FORCE)
+set(KOKKOS_CXXLDFLAGS -L/usr/local/cuda/lib64 CACHE STRING "" FORCE)
+set(KOKKOS_LIBS -lkokkos -ldl -lrt -lcudart -lcuda CACHE STRING "" FORCE)
+set(KOKKOS_EXTRA_LIBS -ldl -lrt -lcudart -lcuda CACHE STRING "" FORCE)
 set(KOKKOS_LINK_FLAGS -fopenmp CACHE STRING "extra flags to the link step (e.g. OpenMP)" FORCE)

 #Internal settings which need to propagated for Kokkos examples
@@ -45,9 +45,9 @@ set(KOKKOS_INTERNAL_USE_SERIAL 1 CACHE STRING "" FORCE)
 set(KOKKOS_INTERNAL_USE_ROCM 0 CACHE STRING "" FORCE)
 set(KOKKOS_INTERNAL_USE_QTHREADS 0 CACHE STRING "" FORCE)
 set(KOKKOS_SRC /scratch/kokkos/core/src/impl/Kokkos_Serial.cpp /scratch/kokkos/core/src/impl/Kokkos_ExecPolicy.cpp /scratch/kokkos/core/src/impl/Kokkos_HostThreadTeam.cpp /scratch/kokkos/core/src/impl/Kokkos_Serial_Task.cpp /scratch/kokkos/core/src/impl/Kokkos_SharedAlloc.cpp /scratch/kokkos/core/src/impl/Kokkos_Spinwait.cpp /scratch/kokkos/core/src/impl/Kokkos_Profiling_Interface.cpp /scratch/kokkos/core/src/impl/Kokkos_CPUDiscovery.cpp /scratch/kokkos/core/src/impl/Kokkos_hwloc.cpp /scratch/kokkos/core/src/impl/Kokkos_HostBarrier.cpp /scratch/kokkos/core/src/impl/Kokkos_Error.cpp /scratch/kokkos/core/src/impl/Kokkos_Core.cpp /scratch/kokkos/core/src/impl/Kokkos_HostSpace.cpp /scratch/kokkos/core/src/impl/Kokkos_MemoryPool.cpp /scratch/kokkos/containers/src/impl/Kokkos_UnorderedMap_impl.cpp /scratch/kokkos/core/src/Cuda/Kokkos_Cuda_Locks.cpp /scratch/kokkos/core/src/Cuda/Kokkos_CudaSpace.cpp /scratch/kokkos/core/src/Cuda/Kokkos_Cuda_Task.cpp /scratch/kokkos/core/src/Cuda/Kokkos_Cuda_Impl.cpp /scratch/kokkos/core/src/OpenMP/Kokkos_OpenMP_Task.cpp /scratch/kokkos/core/src/OpenMP/Kokkos_OpenMP_Exec.cpp CACHE STRING "Kokkos source list" FORCE)
-set(KOKKOS_CXX_FLAGS -I./ -I/scratch/kokkos/core/src -I/scratch/kokkos/containers/src -I/scratch/kokkos/algorithms/src -I/scratch/kokkos/core/src/eti --std=c++11 -Xcudafe --diag_suppress=esa_on_defaulted_function_ignored -expt-extended-lambda -mavx -arch=sm_70 -I/usr/local/cuda/include -Xcompiler -fopenmp)
+set(KOKKOS_CXX_FLAGS --std=c++11 -Xcudafe --diag_suppress=esa_on_defaulted_function_ignored -expt-extended-lambda -mavx -arch=sm_70 -Xcompiler -fopenmp)
 set(KOKKOS_CPP_FLAGS )
-set(KOKKOS_LD_FLAGS -L/scratch/kokkos/build/core -mavx -arch=sm_70 -L/usr/local/cuda/lib64 -fopenmp)
-set(KOKKOS_LIBS_LIST "-lkokkos -ldl -lcudart -lcuda")
-set(KOKKOS_EXTRA_LIBS_LIST "-ldl -lcudart -lcuda")
+set(KOKKOS_LD_FLAGS -mavx -arch=sm_70 -L/usr/local/cuda/lib64 -fopenmp)
+set(KOKKOS_LIBS_LIST "-lkokkos -ldl -lrt -lcudart -lcuda")
+set(KOKKOS_EXTRA_LIBS_LIST "-ldl -lrt -lcudart -lcuda")
 set(KOKKOS_LINK_FLAGS -fopenmp)
```

# TL;DR

But we explicitly configured with `-D CMAKE_BUILD_TYPE=Release`!?

(If we didn't specify anything empty which yields no debug nor optimization flags will be used)

`-D KOKKOS_ENABLE_DEBUG=<ON|OFF>`???

`KOKKOS_CXX_FLAGS` has include directories when installed with generated makefile

What about `KOKKOS_CXX_STANDARD`?  `"c++11"`  
Supported values for `CMAKE_CXX_STANDARD` are `98`, `11`, `14`, `17`, and `20`

* Duplicated `KOKKOS_LINK_FLAGS`

* `KOKKOS_CXXFLAGS`
`KOKKOS_CPPFLAGS`
`KOKKOS_LDFLAGS`
`KOKKOS_CXXLDFLAGS`
 next to 
`KOKKOS_CXX_FLAGS`
`KOKKOS_CPP_FLAGS`
`KOKKOS_LD_FLAGS`

* Not following CMake convention `<PakageName>_`


# What about targets?

* CMake defines a `kokkos` target (case sensitive!)

* You are on your own if you installed using a generated makefile

# How to find Kokkos and load its settings

Essentially two options
* Either use what's already there (exported target from Kokkos built using CMake)
* Or do it yourself (find Kokkos module)

## OPTION A: Use configuration exported by CMake build 

```CMake
find_package(Kokkos REQUIRED)
target_link_libraries(<target> kokkos) # case sensitive (k)okkos
target_include_directories(<target> PUBLIC ${Kokkos_INCLUDE_DIRS}) 
```

### Caevats
* need to add `<prefix>/lib/CMake/Kokkos` to `CMAKE_PATH_PREFIX`

* tell your users they have to build Kokkos using cmake

## OPTION B: Write and maintain a Find Module

Write a "find module", i.e. a `FindKokkos.cmake` file to be loaded by `find_package()` when invoked for `Kokkos`

* Traditional approach is to use variables for everything, including libraries and executables

`Kokkos_FOUND`
`Kokkos_INCLUDE_DIRS`
`Kokkos_LIBRARIES`
`Kokkos_DEFINITIONS`
`Kokkos_LIBRARY_DIRS`
`Kokkos_VERSION`

`Kokkos_NVCC_WRAPPER_EXECTUABLE`

* More modern approach is to behave as much like config file packages files as possible, by providing **imported target**.  
This has the advantage of propagating transitive usage requirements to consumers.

May use cache variables for users to edit and control the behaviour of the find module

May provide functions or macros

## Find the Kokkos library

Try to use `pkg-config` to find the library since it is currently available for Kokkos

```CMake
find_package(PkgConfig)
pkg_check_modules(PC_Kokkos QUIET kokkos)
```
This will define some variables starting `PC_Kokkos_` that contain the information from the `kokkos.pc` file


Use the information from `pkg-config` to provide hints to CMake about where to look
```CMake
find_path(Kokkos_INCLUDE_DIR
  NAMES Kokkos_Core.hpp
  PATHS ${PC_Kokkos_INCLUDE_DIRS}
)

find_library(Kokkos_LIBRARY
  NAMES kokkos
  PATHS ${PC_Kokkos_LIBRARY_DIRS}
)
```


Get the version somehow...
```CMake
set(Kokkos_VERSION ${PC_Kokkos_VERSION}) # not currently defined in kokkos.pc
```

Use `FindPackageHandleStandardArgs` to do most of the rest of the work
```CMake
include(FindPackageHandleStandardArgs)
find_package_handle_standard_args(Kokkos
  FOUND_VAR Kokkos_FOUND
  REQUIRED_VARS
    Kokkos_LIBRARY
    Kokkos_INCLUDE_DIR
  VERSION_VAR Kokkos_VERSION 
)
```

## Provide a way for users of the find module to link to Kokkos

Traditional variable approach looks like
```CMake
if(Kokkos_FOUND)
  set(Kokkos_LIBRARIES ${Kokkos_LIBRARY})
  set(Kokkos_INCLUDE_DIRS ${Kokkos_INCLUDE_DIR})
  set(Kokkos_DEFINITIONS ${PC_Kokkos_CFLAGS_OTHER})
endif()
```

Modern approach is
```CMake
if(Kokkos_FOUND AND NOT TARGET Kokkos::Kokkos)
  add_library(Kokkos::Kokkos UNKNOWN IMPORTED)  
  set_target_properties(Kokkos::Kokkos PROPERTIES
    IMPORTED_LOCATION "${Kokkos_LIBRARY}"
    INTERFACE_COMPILE_OPTIONS "${PC_Kokkos_CFLAGS_OTHER}"
    INTERFACE_INCLUDE_DIRECTORIES "${Kokkos_INCLUDE_DIR}"
  )
endif()
```

When providing imported target, these should be namespaced (note the `Kokkos::` prefix)  
CMake will recognize that values passed to `target_link_libraries()` that contain `::` in their name are supposed to be imported targets (rather than just library names), and will produce appropriate diagnostic messages if that target does not exist 

```CMake
#[=======================================================================[.rst:
FindKokkos
-------

Finds the Kokkos library.

Imported Targets
^^^^^^^^^^^^^^^^

This module provides the following imported targets, if found:

``Kokkos::Kokkos``
  The Kokkos library

Result Variables
^^^^^^^^^^^^^^^^

This will define the following variables:

``Kokkos_FOUND``
  True if the system has the Kokkos library.
``Kokkos_VERSION``
  The version of the Kokkos library which was found.
``Kokkos_INCLUDE_DIRS``
  Include directories needed to use Kokkos.
``Kokkos_LIBRARIES``
  Libraries needed to link to Kokkos.

Cache Variables
^^^^^^^^^^^^^^^

The following cache variables may also be set:

``Kokkos_INCLUDE_DIR``
  The directory containing ``Kokkos_Core.hpp``.
``Kokkos_LIBRARY``
  The path to the Kokkos library.

#]=======================================================================]

find_package(PkgConfig)
pkg_check_modules(PC_Kokkos QUIET kokkos)

find_path(Kokkos_INCLUDE_DIR
  NAMES Kokkos_Core.hpp
  PATHS ${PC_Kokkos_INCLUDE_DIRS}
)
find_library(Kokkos_LIBRARY
  NAMES kokkos
  PATHS ${PC_Kokkos_LIBRARY_DIRS}
)

set(Kokkos_VERSION ${PC_Kokkos_VERSION})

include(FindPackageHandleStandardArgs)
find_package_handle_standard_args(Kokkos
  FOUND_VAR Kokkos_FOUND
  REQUIRED_VARS
    Kokkos_LIBRARY
    Kokkos_INCLUDE_DIR
  VERSION_VAR Kokkos_VERSION 
)

if(Kokkos_FOUND)
  set(Kokkos_LIBRARIES ${Kokkos_LIBRARY})
  set(Kokkos_INCLUDE_DIRS ${Kokkos_INCLUDE_DIR})
  set(Kokkos_DEFINITIONS ${PC_Kokkos_CFLAGS_OTHER})
endif()

if(Kokkos_FOUND AND NOT TARGET Kokkos::Kokkos)
  add_library(Kokkos::Kokkos UNKNOWN IMPORTED)  
  set_target_properties(Kokkos::Kokkos PROPERTIES
    IMPORTED_LOCATION "${Kokkos_LIBRARY}"
    INTERFACE_COMPILE_OPTIONS "${PC_Kokkos_CFLAGS_OTHER}" # !!!!
    INTERFACE_INCLUDE_DIRECTORIES "${Kokkos_INCLUDE_DIR}"
  )
endif()

mark_as_advanced(
  Kokkos_INCLUDE_DIR
  Kokkos_LIBRARY
)
```

## OPTION C: Let pkg-config create an imported target?

```CMake
find_package(PkgConfig REQUIRED)
pkg_check_modules(PC_Kokkos REQUIRED IMPORTED_TARGET kokkos)
target_link_libraries(<target> PkgConfig::PC_Kokkos)
target_link_libraries(<target> ${PC_Kokkos_LDFLAGS}) # sigh...
```

# Have Kokkos always export its target?

# What are the risks and the payoffs?


Break everybody's working build system

```cmake
find_package(Kokkos REQUIRED)
target_link_libraries(<target> Kokkos::kokkos)
```



# NONAME

Every project that uses Kokkos write its own `FindKokkos.cmake`?

```CMake
#[=======================================================================[.rst:

This function makes sure that kokkos was built with the requested backends
and target architectures and generates a fatal error if it was not.

kokkos_check_requested(
  [DEVICES <devices>...] # Set of backends (e.g. "OpenMP" and/or "Cuda")
  [ARCH <archs>...]      # Target architectures (e.g. "Power9" and/or "Volta70")
)
#]=======================================================================]
function(kokkos_check_requested)
  cmake_parse_arguments(_KOKKOS_REQUESTED "" "" "DEVICES;ARCH" ${ARGN})
  set(_KOKKOS_REQUESTED_ARGS)
  foreach(_X DEVICES ARCH)
    if(_KOKKOS_REQUESTED_${_X})
      list(APPEND _KOKKOS_REQUESTED_ARGS ${_X})
    endif()
  endforeach()
  set(_KOKKOS_CHECK_REQUESTED_SUCCESS TRUE)
  foreach(_X ${_KOKKOS_REQUESTED_ARGS})
    foreach(_requested ${_KOKKOS_REQUESTED_${_X}})
      foreach(_provided ${KOKKOS_${_X}})
        if(_requested STREQUAL _provided)
          string(REPLACE ";" " " ${_requested} "${_KOKKOS_REQUESTED_${_X}}")
        endif()
      endforeach()
    endforeach()
    find_package_handle_standard_args("KOKKOS_${_X}" DEFAULT_MSG ${_KOKKOS_REQUESTED_${_X}})
    if(NOT KOKKOS_${_X}_FOUND)
      set(_KOKKOS_CHECK_REQUESTED_SUCCESS FALSE)
    endif()
  endforeach()
  if(NOT _KOKKOS_CHECK_REQUESTED_SUCCESS)
    message(FATAL_ERROR "Kokkos does NOT provide all backends and/or architectures requested")
  endif()
endfunction()
```

# Summary

What do we want

1. Install a new package configuration file `<prefix>/lib/cmake/kokkos-config.cmake` 
2. Export namespaced targets `Kokkos::kokkos` alongside the pre-existing one to avoid breaking everybody's code
3. Inject deprecation warning in `<prefix>/lib/CMake/Kokkos/KokkosConfig.cmake` (?)
4. Function to check at configure time that the Kokkos that was found has desirable properties
5. Consider providing a `FindKokkos.cmake` module that provide similar functionality for backwards compatibility (?)

Broken with clang